<a href="https://colab.research.google.com/github/GodJiLee/Machine_learning_study/blob/master/9%EC%9E%A5_%EC%9B%B9_%EC%95%A0%ED%94%8C%EB%A6%AC%EC%BC%80%EC%9D%B4%EC%85%98_%EA%B0%9C%EB%B0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9장 웹 애플리케이션에 머신러닝 모델 내장

## 8장 복습하기

In [0]:
!wget https://github.com/rickiepark/python-machine-learning-book-2nd-edition/raw/master/code/ch09/movie_data.csv.gz

--2020-06-08 05:01:55--  https://github.com/rickiepark/python-machine-learning-book-2nd-edition/raw/master/code/ch09/movie_data.csv.gz
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-2nd-edition/master/code/ch09/movie_data.csv.gz [following]
--2020-06-08 05:01:55--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-2nd-edition/master/code/ch09/movie_data.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26521894 (25M) [application/octet-stream]
Saving to: ‘movie_data.csv.gz’

movie_data.csv.gz   100%[===================>]  25.29M  3

In [0]:
import gzip

with gzip.open('movie_data.csv.gz') as f_in, open('movie_data.csv', 'wb') as f_out:
    f_out.writelines(f_in)

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [0]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [0]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [0]:
!pip install pyprind

In [0]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:27


In [0]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('정확도: %.3f' % clf.score(X_test, y_test))

정확도: 0.868


In [0]:
clf = clf.partial_fit(X_test, y_test)

## 9.1 학습된 사이킷런 추정기 저장

In [0]:
import pickle
import os
dest = os.path.join('movieclassifier', 'pkl_objects')

if not os.path.exists(dest):
    os.makedirs(dest) # 데이터를 저장할 디렉터리 생성

pickle.dump(stop, # 자연어 라이브러리의 불용어 저장 기능
            open(os.path.join(dest, 'stopwords.pkl'), 'wb'), # wb: 이진 모드로 파일 오픈
            protocol = 4) # 파이썬 3.4 버전에 최적화

pickle.dump(clf, 
            open(os.path.join(dest, 'classifier.pkl'), 'wb'),
            protocol = 4)

In [0]:
%%writefile movieclassifier/vectorizer.py

from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(os.path.join(cur_dir,
                                     'pkl_object',
                                     'stopwords.pkl'), 'rb'))
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emotions = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                          text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]

    return tokenized

# 토큰형 행렬로 문서를 변환해주는 메서드
vect = HashingVectorizer(decode_error = 'ignore',
                         n_features = 2 * 21,
                         preprocessor = None,
                         tokenizer = tokenizer)

Writing movieclassifier/vectorizer.py


In [0]:
import os
os.chdir('movieclassifier') # 디렉터리 위치 변경

In [0]:
import numpy as np
label = {0: '음성', 1: '양성'}

example = ['I love this movie']

X = vect.transform(example) # HashingVectorizer를 사용해 example을 벡터로 변환 
print('예측: %s\n 확률: %.2f%%' %\
      (label[clf.predict(X)[0]], # 클래스 레이블 예측
       np.max(clf.predict_proba(X))*100)) # 예측의 확률값 계산

예측: 양성
 확률: 81.44%


## 9.2 데이터를 저장하기 위해 SQLite 데이터베이스 설정

In [0]:
import sqlite3

conn = sqlite3.connect('reviews.sqlite') # SQLite 데이터베이스 파일에 연결
c = conn.cursor() # 데이터베이스 커서 생성 (DB 문법 사용 가능)

c.execute('DROP TABLE IF EXISTS review_db')
c.execute('CREATE TABLE review_db'\
          '(review TEXT, sentiment INTEGER, date TEXT)') # 컬럼 생성

example1 = 'I love this movie'
c.execute('INSERT INTO review_db'\
          "(review, sentiment, date) VALUES"\
          "(?, ?, DATETIME('now'))", (example1, 1))

example2 = 'I dislike this movie'
c.execute("INSERT INTO review_db"\
          "(review, sentiment, date) VALUES"\
          "(?, ?, DATETIME('now'))", (example2, 0))

conn.commit() # 정상적으로 실행시 변경사항 저장
conn.close() # 종료

In [0]:
conn = sqlite3.connect('reviews.sqlite') # 재연결
c = conn.cursor()
c.execute("SELECT * FROM review_db WHERE date"\
          " BETWEEN '2017-01-01 00:00:00' AND DATETIME('now')") # 2017년부터 오늘까지 추가된 모든 열 추출

results = c.fetchall()
conn.close()

print(results)

[('I love this movie', 1, '2020-06-08 05:05:17'), ('I dislike this movie', 0, '2020-06-08 05:05:17')]
